In [4]:
import postgresql
from random import randint
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import os
import sys


In [5]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.psql import Postresql

# Конфигурации

In [6]:
database = Postresql({
    'database': 'fishtexts',
    'user': 'postgres',
    'password': '081099',
    'host': 'localhost',
    'port': '5432'
})

# Сбор данных

In [7]:
comments = pd.DataFrame(database.select('comments'))
comments

,mini_place,main_place,date,marked,text,id
0,Брейтово,Рыбинское водохранилище.,03.02.2015,yes,3 февраль вторник . утро с афоня раз...,1
1,Брейтово,Рыбинское водохранилище.,06.02.2015,yes,"6 февраль , пятница . подъем в 5 утро ...",2
2,Брейтово,Рыбинское водохранилище.,09.02.2015,yes,"9 февраль , понедельник . самый результати...",3
3,Брейтово,Рыбинское водохранилище.,11.02.2015,yes,"вчерашний золотой лунка молчать , как ...",4
4,Брейтово,Рыбинское водохранилище.,03.03.2015,yes,3 марта вторник . тот же район море ...,5
...,...,...,...,...,...,...
85905,Брейтово,Рыбинское водохранилище.,28.07.2016,yes,четверг 3 час утро 28 число прини...,85906
85906,Брейтово,Рыбинское водохранилище.,24.02.2014,yes,"вчера , тем мало мы она поймать зав...",85907
85907,Брейтово,Рыбинское водохранилище.,01.03.2014,yes,сегодня половить рыбка . выезжать от ч...,85908
85908,Брейтово,Рыбинское водохранилище.,03.03.2014,yes,3 по 6.03 . ловить на жерлица в ...,85909


# Обработка

In [8]:
predators_comments = pd.DataFrame(database.select('хищник'))
peace_comments = pd.DataFrame(database.select('мирная'))
pike_comments = pd.DataFrame(database.select('щука'))
sudak_comments = pd.DataFrame(database.select('судак'))
catfish_comments = pd.DataFrame(database.select('сом'))
forel_comments = pd.DataFrame(database.select('форель'))
okun_comments = pd.DataFrame(database.select('окунь'))
golavl_comments = pd.DataFrame(database.select('голавль'))
gereh_comments = pd.DataFrame(database.select('жерех'))
yaz_comments = pd.DataFrame(database.select('язь'))
carp_comments = pd.DataFrame(database.select('карп'))
caras_comments = pd.DataFrame(database.select('карась'))
plotva_comments = pd.DataFrame(database.select('плотва'))
lesh_comments = pd.DataFrame(database.select('лещ'))
krasnoperka_comments = pd.DataFrame(database.select('красноперка'))
gustera_comments = pd.DataFrame(database.select('густера'))
tolstolobik_comments = pd.DataFrame(database.select('толстолобик'))
lin_comments = pd.DataFrame(database.select('линь'))
comments = pd.DataFrame(database.select('comments'))


In [9]:
fish_comments = pd.DataFrame(columns=['mini_place', 'date', 'marked', 'main_place', 'text' ,'fish', 'p_dost', 'p_fastai'])
types_comments = pd.DataFrame(columns=['mini_place', 'date', 'marked', 'main_place', 'text' ,'type', 'p_dost', 'p_fastai'])

fishs = ['щука', 'судак', 'сом', 'форель', 'окунь', 'голавль', 'жерех', 'язь',
         'карп', 'карась', 'плотва', 'лещ', 'красноперка', 'густера', 'толстолобик', 'линь']
types = ['хищник', 'мирная']

In [10]:
for fish in fishs:
    temp_dt = pd.DataFrame(database.select(fish))
    temp_dt['fish'] = fish
    del temp_dt['id']
    fish_comments = fish_comments.append(temp_dt, ignore_index=True)
    
for t in types:
    temp_dt = pd.DataFrame(database.select(t))
    temp_dt['type'] = t
    del temp_dt['id']
    types_comments = types_comments.append(temp_dt, ignore_index=True)

C:\Users\Xiaomi\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
stop_words = set(stopwords.words('russian'))
stop_words = stop_words.intersection('не')
replace_word_map = {'ноль': 'очень плохо', 'вяло': 'плохо', 'вялый': 'плохой',
                    'молчать': 'плохо ловиться', 'молчать': 'клевать плохо', 'судак': 'рыба', 'судачок': 'рыба',
                    'опять': '', 'елец': 'рыба', 'жор': 'классно великолепно', 'сопа': 'рыба', 'подлец': 'рыба',
                   'ловиться': 'результат', 'несколько': '', 'тишина': 'плохо'}
replace_sentence = {'полный ноль': 'ужасно', 'не густо': 'плохо', 'рыбалка удаваться': 'рыбалка отлично',
                    'не быть': 'было мало', 'не плох': 'хорош'}

def preprocessing(text):
    temp = text.lower()
    temp = re.sub(r'\d+', '', temp)
    temp = temp.translate(str.maketrans('','', string.punctuation)).strip()
    tokens = word_tokenize(temp)
    temp = [i.strip() for i in tokens if not i in stop_words]
    temp = [replace_word_map[i] if i in replace_word_map else i for i in temp]
    temp = [i for i in temp if len(i) > 1]
    temp = ' '.join(temp)
    for sentence in replace_sentence:
        temp = temp.replace(sentence, replace_sentence[sentence])
    return temp

In [ ]:
for index, row in tqdm_notebook(fish_comments.iterrows()):
    fish_comments['text'][index] = preprocessing(row['text'])

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [ ]:
for index, row in tqdm_notebook(types_comments.iterrows()):
    types_comments['text'][index] = preprocessing(row['text'])

In [ ]:
fish_comments

# Предсказания

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

model = FastTextSocialNetworkModel(tokenizer=RegexTokenizer())


In [ ]:
for index, row in tqdm_notebook(fish_comments.iterrows()):
    pred = model.predict([row['text']], k=5)[0]
    if pred['skip'] > 0.5:
        fish_comments['p_dost'][index] = np.NaN
    elif pred['neutral'] > 0.85:
        fish_comments['p_dost'][index] = np.NaN
    elif pred['negative'] < 0.1 and pred['positive'] < 0.1:
        fish_comments['p_dost'][index] = np.NaN
    else:
        fish_comments['p_dost'][index] = 1 - pred['negative']

In [ ]:
for index, row in tqdm_notebook(types_comments.iterrows()):
    pred = model.predict([row['text']], k=5)[0]
    if pred['skip'] > 0.5:
        types_comments['p_dost'][index] = np.NaN
    elif pred['neutral'] > 0.85:
        types_comments['p_dost'][index] = np.NaN
    elif pred['negative'] < 0.1 and pred['positive'] < 0.1:
        types_comments['p_dost'][index] = np.NaN
    else:
        types_comments['p_dost'][index] = 1 - pred['negative']

In [ ]:
del fish_comments['p_fastai']
del types_comments['p_fastai']

In [ ]:
isna_indexes_comments = fish_comments[fish_comments.p_dost.isna()].index
isna_indexes_types = types_comments[types_comments.p_dost.isna()].index

In [ ]:
preds_comments = [0.75 if _ == np.nan else _ for _ in fish_comments.p_dost]
preds_types = [0.75 if _ == np.nan else _ for _ in types_comments.p_dost]

fish_comments['p_dost'] = preds_comments
types_comments['p_dost'] = preds_types

# Обработка мест ловли

In [ ]:
column_main_place = []
for index, row in tqdm_notebook(fish_comments.iterrows()):
    if row['main_place'].find('�?')>-1:
        column_main_place.append(row['main_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['main_place'])
fish_comments['main_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(fish_comments.iterrows()):
    if row['mini_place'].find('�?')>-1:
        column_main_place.append(row['mini_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['mini_place'])
fish_comments['mini_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(types_comments.iterrows()):
    if row['main_place'].find('�?')>-1:
        column_main_place.append(row['main_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['main_place'])
types_comments['main_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(types_comments.iterrows()):
    if row['mini_place'].find('�?')>-1:
        column_main_place.append(row['mini_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['mini_place'])
types_comments['mini_place'] = column_main_place




In [ ]:
places = set()
for index, row in tqdm_notebook(fish_comments.iterrows()):
    places.add(' '.join([row['main_place'], row['mini_place']]))
places

In [ ]:
from google import google
num_page = 1
search_results = google.search("Рыбинское водохранилище. Брейтово", num_page)

In [ ]:
print(search_results[0].link)

In [ ]:
from deeppavlov import configs, build_model

ner_model = build_model('ner_rus_bert')

In [ ]:
def city_name_find(text):
    predict_result = ner_model([text])
    result = []
    for index in range(len(predict_result[1][0])):
        if predict_result[1][0][index] in ['B-LOC', 'I-LOC']:
            result.append(predict_result[0][0][index])
    return ' '.join(result)

In [ ]:
places_dict = {place: city_name_find(place) for place in tqdm_notebook(places)}

In [ ]:
search_results = {}
for key in tqdm_notebook(places_dict):
    if places_dict[key]:
        search_results[key] = google.search(places_dict[key] + ' википедия', 1)
    else:
        search_results[key] = []

In [ ]:
for key in tqdm_notebook(places_dict):
    if places_dict[key]:
        if not search_results[key]:
            search_results[key] = google.search(places_dict[key] + ' википедия', 1)

In [ ]:
import requests
from time import sleep

In [ ]:
wiki_pages = {}
for key in tqdm_notebook(search_results):
    temp = search_results[key]
    for page in temp:
        sleep(1)
        link = page.link
        if not link:
            continue
        page_source = requests.get(link)
        if 'Субъект Федерации' in page_source.text or 'Субъект РФ' in page_source.text or 'Регионы' in page_source.text:
            wiki_pages[key] = page_source
            break

In [72]:
page_source.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="ru" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Богослово (Щёлковский район) — Википедия</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgMonthNamesShort":["","янв","фев","мар","апр","май","июн","июл","авг","сен","окт","ноя","дек"],"wgRequestId":"XmlVtQpAADwAAJhCy2UAAADB","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Богослово_(Щёлковский_район)","wgTitle":"Богослово (Щёлковский район)","wgCurRevisionId":104582979,"wgRevisionId":104582979,"wgArticleId":2608536,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[\n"Википедия:Статьи с некор